# 📊 **Supply Chain Performance — Power BI Dashboard**

>This section of the project showcases the design and implementation of a **Power BI dashboard** built to analyze and visualize supply chain data from the [DataCo Smart Supply Chain dataset](https://data.mendeley.com/datasets/8gx2fvg2k6/5).  
>The objective is to deliver **executive-level insights** through dynamic KPIs, trend analysis and segmentation tools that support **operations**, **logistics** and **customer management**.

**Key focus areas include:**
- `Order Profitability`
- `Delivery Efficiency and Delays`
- `Customer Segmentation`
- `Product Performance by Category and Geography`

## 🧩 **Data Modeling Recap**

To enable efficient reporting and slicing in Power BI, the dataset was transformed into a **Snowflake Schema**, with a centralized `fact_orders` table and supporting dimension tables.

The data was exported from BigQuery (after cleaning and normalization) and imported into Power BI as a set of CSV files.

**Relationship Diagram:**
- 📎 View model on [dbdiagram.io](https://dbdiagram.io/d/DataCo-Supply-Chain-Schem-68308545b9f7446da3e05968)
- 📎 Interactive docs on [dbdocs.io](https://dbdocs.io/bsilva1297/DataCo-Supply-Chain-Schema?view=table_structure)
- 📎 PDF version in [GitHub Repo](https://github.com/brunopata/DataCo-Supply-Chain/blob/main/schema/DataCo%20Supply%20Chain%20Tables%20Schema.pdf)

### 🔗 Table Summary

#### 📌 *Fact Table:*

- `fact_orders` – Central transactional table with **quantity**, **revenue**, **dates**, **shipping** and **customer/product** linkage

#### 📎 *Dimension Tables:*

- `dim_customer` – Customer profile, region, segment
- `dim_product` – Product details, price, status
- `dim_category` – Product category and department
- `dim_shipping` – Delivery status and timeliness
- `dim_order_location` – Order origin geography
- `dim_date` – Calendar table used for date hierarchy and trends

#### 🧮 *Relationships* 

The following key principles and relationships were created in the Power BI model to support star/snowflake schema analysis. These connections enable slicing and filtering between related tables across time, geography, customer behavior and product sales.

🔑 **Key relationship principles used:**
- **One-to-Many (1:M)** joins: All dimension tables have unique primary keys (e.g. `customer_id`, `product_id`) that link to repeating foreign keys in the fact table.
- **Date Handling**: The `dim_date` table is used twice — for both `order_date` and `shipping_date`. A secondary inactive relationship is used for `shipping_date` and activated in DAX using the `USERELATIONSHIP()` function where needed.
- **Granularity**: The fact table is at the **order_level**, while all dimensions represent unique attributes (customer, product, date, etc.)
- **Shipping and Location**: Linked by `order_id` — one row per order in `dim_shipping` and `dim_order_location`.

> These relationships are crucial to enable **filter propagation** **drill-down analysis** and **accurate aggregation** in the report visuals.

🔄 **Relationship Summary:**
- `fact_orders.customer_id` ➝ `dim_customer.customer_id` — Many-to-One _(M:1)_
- `fact_orders.product_id` ➝ `dim_product_id` — Many-to-One _(M:1)_
- `dim_product.category_id` ➝ `dim_category.category_id` — Many-to-One _(M:1)_
- `fact_orders.order_id` ➝ `dim_shipping.order_id` — Many-to-One _(M:1)_
- `fact_orders.order_id` ➝ `dim_order_location.order_id` — Many-to-One _(M:1)_
- `fact_orders.order_date` ➝ `dim_date.date` — Many-to-One _(M:1)_ _(Active)_
- `fact_orders.shipping_date` ➝ `dim_date.date` _(inactive, used with `USERELATIONSHIP`)_ — Many-to-One _(M:1)_ _(Inactive)_

## 💡 **Executive KPI Metrics**

> These KPI measures will power the executive overview page of the dashboard, allowing leadership to track business performance at a glance while enabling further exploration by region, customer or time.

### 🧾 Total Orders

**🎯 Business Purpose:**  
Measures the total number of orders placed. This is the foundational metric for volume-based reporting and revenue tracking.

In [ ]:
```DAX

Total Orders = DISTINCTCOUNT(DataCoSC_fact_orders[order_id])

> This KPI can be used to monitor overall growth, order frequency and to benchmark across time or regions.

### 📌 Average Orders

🎯 Business Purpose:
Calculates the average number of orders and shows performance trends over time.

In [ ]:
```DAX

Average Orders =
DIVIDE(
    DISTINCTCOUNT(DataCoSC_fact_orders[order_id]),
    DISTINCTCOUNT(DataCoSC_dim_date[year_month])
)

### 💰 Total Revenue

**🎯 Business Purpose:**  
Captures the total sales generated from all orders. Core business performance metric.

In [ ]:
```DAX

Total Revenue = SUM(DataCoSC_fact_orders[item_total])

> Compared across time to monitor growth. Use with profit to evaluate margins.

### 💸 Total Profit

**🎯 Business Purpose:**  
Shows earnings after subtracting costs per order. Measures **true** business gain.

In [ ]:
```DAX

Total Profit = SUM(DatacoSC_fact_orders[order_profit_per_order])

> Used to track net business performance and identify product or regional profit gaps.

### 💲 Average Profit

**🎯 Business Purpose:**  
Calculates average profit value.

In [ ]:
```DAX

Average Profit = SUM(DataCoSC_fact_orders[order_profit_per_order]) / COUNT(DataCoSC_fact_orders[order_id])

> Lower values indicate that the business is not profitable, maybe by the use of heavy discounting or low margins.

### 💯 Discount Orders

**🎯 Business Purpose:**  
Calculates number of orders that are discounted.

In [ ]:
```DAX

Discount Orders =
CALCULATE(
    [Total Orders],
    DataCoSC_fact_orders[item_discount] > 0
)

> Track number of orders that are discounted. Used to calculate the rate of orders that are discounted.

### 🔖 Total Discount

**🎯 Business Purpose:**  
Calculates total disount value through all orders and products.

In [ ]:
```DAX

Total Discount = SUM(DatacoSC_fact_orders[item_discount])

> Used to track total discount. Used to check if there are any heavy discounting that leads to loss of profit.

### 🏷️ Average Discount

**🎯 Business Purpose:**  
Calculates average discount value.

In [ ]:
```DAX

Average Discount = SUM(DataCoSC_fact_orders[item_discount]) / COUNT(DataCoSC_fact_orders[order_id])

> Higher values may indicate discounts are high, which may lead to major loss of profit.

### 🎟️ Discount Orders Rate

**🎯 Business Purpose:**  
Calculates rate of orders that are discounted.

In [ ]:
```DAX

Discount Orders Rate = ([Discount Orders] / [Total Orders]) * 100

> Aiming for a discount rate of 5%-15% is a good starting point — especially if margins are on the lower side.

### 📈 Profit Margin

**🎯 Business Purpose:**  
Evaluates profitability efficiency by comparing revenue vs profit.

In [ ]:
```DAX

Profit Margin (%) = ([Total Profit] / [Total Revenue]) * 100

> Monitors erosion — low margins may result from discounts or fulfillment costs.

### 🛒 Average Cart Size

**🎯 Business Purpose:**  
Measures average quantity of items purchased per order.

In [ ]:
```DAX

Average Cart Size =
AVERAGEX(
    VALUES(DataCoSC_fact_orders[order_id]),
    SUM(DataCoSC_fact_orders[item_quantity])
)

> Getting higer values may indicate strong bundling, volume purchases or good upselling.

### 🚚 Average Shipping Delay

**🎯 Business Purpose:**  
Compares actual vs. scheduled delivery time. Evaluates operational efficiency.

In [ ]:
```DAX

Avg Shipping Delay = 
AVERAGEX(
    VALUES(DataCoSC_fact_orders[order_id]),
    CALCULATE(
        MAXX(
            DataCoSC_fact_orders,
            DataCoSC_fact_orders[days_for_shipping] - DataCoSC_fact_orders[days_for_shipment_scheduled]
        )
    )
)

> Even short delays (avg. >1 day) can impact SLAs and customer satisfaction.

### 📦 Late Delivery Orders

**🎯 Business Purpose:**  
Calculates the total of orders delivered late.

In [ ]:
```DAX

Late Delivery Orders = 
CALCULATE(
    [Total Orders],
    DataCoSC_fact_orders[late_delivery] = 1
)

### ⏱ Late Delivery Rate

**🎯 Business Purpose:**  
Calculates the percentage of orders delivered late.

In [ ]:
```DAX

Late Delivery Rate (%) = ([Late Delivery Orders] / [Total Orders]) * 100

> Should be aimed for <10%. High rate will require investigation into fulfillment timing or carries issues.

### 👤 Total Customers

**🎯 Business Purpose:**  
Show high-level customer performance trends over time.

In [ ]:
```DAX

Total Customers = DISTINCTCOUNT(DataCoSC_fact_orders[customer_id])

### 💡 Specific KPIs

#### _New vs. Returning Customers_

**🎯 Business Purpose:**  
Enables strategic depth and a clearer view of acquisition vs retention of customers.

Customers will be classified as:
- **New** → first time this customer made a purchase
- **Returning** → any subsequent purchase

##### **1. Create First Order Date Table (Create Table)**

In [ ]:
```DAX

customer_first_orders =
ADDCOLUMNS(
    VALUES(DataCoSC_fact_orders[customer_id]),
    "first_order_date",
    CALCULATE(
        MIN(DataCoSC_fact_orders[order_date])
    )
)

##### **2. New vs. Returning Customer Classification (Create Column)**

In [ ]:
```DAX

customer_type =
VAR cust_id = customer_first_orders[customer_id]
VAR order_count =
    CALCULATE(
        COUNTROWS(DataCoSC_fact_orders),
        DataCoSC_fact_orders[customer_id] = cust_id
    )
RETURN
IF(order_count = 1, "New", "Returning")

##### **3. KPI Creation**

###### **New Customers**

In [ ]:
```DAX

New Customers = 
CALCULATE(
    DISTINCTCOUNT(DataCoSC_fact_orders[customer_id]),
    customer_first_orders[customer_type] = "Returning"
)

###### **Returning Customers**

In [ ]:
```DAX

New Customers = 
CALCULATE(
    DISTINCTCOUNT(DataCoSC_fact_orders[customer_id]),
    customer_first_orders[customer_type] = "New"
)

###### **% of Returning Customers**

In [ ]:
```DAX

Returning Customer Rate (%) = 
DIVIDE(
    [Returning Customers],
    DISTINCTCOUNT(DataCoSC_fact_orders[customer_id])
) * 100

###### **Revenue by New vs Returning**

In [ ]:
```DAX

Revenue by Customer Type =
CALCULATE(
    SUM(DataCoSC_fact_orders[item_total])
)

# User `customer_type` in the legend to break this into new/returning in visuals

#### _Top Customers Revenue (%)_

**🎯 Business Purpose:**  
Answering **how dependent is the business on top customers**. It enables us to understand if the revenue is concentrated on high-value customers.

In [ ]:
```DAX

Top 10 Customers Contribution (%) = 
VAR TopCustomerRevenue =
    CALCULATE(
        SUM(DataCoSC_fact_orders[item_total]),
        TOPN(10, VALUES(DataCoSC_fact_orders[customer_id]),
            CALCULATE(SUM(DataCoSC_fact_orders[item_total])),
            DESC
        )
    )
VAR TotalRevenue =
    CALCULATE(SUM(DataCoSC_fact_orders[item_total]))

RETURN
DIVIDE(TopCustomerRevenue, TotalRevenue)

#### _Customer Name_

**🎯 Business Purpose:**  
Creates a column for **customer_name (customer_first_name + customer_last_name)** on dim_customer that enables easier identification of customers for segmentation. 

In [ ]:
```DAX

customer_name = DataCoSC_dim_customer[first_name] & " " & DataCoSC_dim_customer[last_name]

#### _Category by Profit_

**🎯 Business Purpose:**  
Highlights top category by profit made.

##### **1. Create Total Profit by Category**

In [ ]:
```DAX

Total Profit by Category = SUM(DataCoSC_fact_orders[order_profit_per_order])

##### **2. Create Top Caregory by Profit**

In [ ]:
```DAX

Top Category by Profit = 
CALCULATE(
    SELECTEDVALUE(DataCoSC_dim_category[category]),
    TOPN(
        1,
        VALUES(DataCoSC_dim_category[category]),
        CALCULATE(SUM(DataCoSC_fact_orders[order_profit_per_order])),
        DESC
    )
)

#### _Negative Profit Orders_

**🎯 Business Purpose:**  
Counts number of orders whose profit value is negative.

In [ ]:
```DAX

Negative Profit Orders =
COUNTROWS(
    FILTER(
        ADDCOLUMNS(
            VALUES(DataCoSC_fact_orders[order_id]),
            "OrderProfit",
            CALCULATE(SUM(DataCoSC_fact_orders[order_profit_per_order]))
        ),
        [OrderProfit] < 0
    )
)

## 📄 **Page 1: Executive Overview Dashboard**

### 🎯 Goal

Deliver a **high-level summary** of operational performance using key business metrics. Designed for decision-makers to get a snapshot of business health across orders, revenue, profit, logistics and customer behavior.

### 🎛 Filters

Global filters enable dynamic exploration:
- 📅 `Date` (from `dim_date`)
- 📦 `Category` (from `dim_product`)
- 🌎 `Country` (from `dim_order_location`)
- 👥 `Segment` (from `dim_customer`)

### 📊 KPIs at the Top

- **Financial KPIs like:**
  - 🧾 Total Orders
  - 💰 Total Revenue
  - 💹 Total Profit
  - 📊 Profit Margin

- **Operational KPIs like:**
  - 🛒 Average Cart Size
  - 📦 Average Shipping Delay
  - ⏱ Late Delivery Rate
  - 🚚 Late Delivery Orders

Each KPI is tied to key insights from the earlier **SQL & DAX analysis**. Measures are filter-responsive for slicing by time, geography or category.

### 📈 Visual Elements 

#### _1. Revenue Over Time_

- **X-axis:** Month/Year (`dim_date[year_month]`)
- **Y-axis:** Total Revenue

🔍 Shows revenue trends over time using a monthly breakdown. Used for spotting seasonality or performance dips.

#### _2. Orders by Country_

- **Location:** Country (`dim_order_location[order_country]`)
- **Gradient:** Uses a conditional color scale to highlight regional total revenue

🔍 A choropleth map that highlights geographical distribution of orders, emphasizing dominant regions and countries.

#### _3. Top Selling Categories_

- **X-axis:** Total Revenue
- **Y-avis:** Category (`dim_category[category]`)
- **Top N:** Uses Top N filtering to highlight Top 10 selling categories by revenue

🔍 Highlights which product categories drive the most revenue, supporting inventory and merketing decisions.

### 🧠 Insights to Highlight

- **Order Voluime:** 65752 orders placed
- **Revenue:** $33M+ in sales, with a healthy 12% profit margin
- **Customer Behavior:** High average cart size of 5.84 items per order
- **Logistics:** Despite fast shipping (avg. delay 0.57 days), 54.82% of deliveries are still marked as late — likely due to unrealistic scheduled times

### 📌 Insights Taken

- 📌**Profit margins** are good but discount strategies and shipping costs should be reviewed
- 📌**High-value customers** are present and should be targeted with loyalty initiatives
- 📌**Cart behavior** suggests volume-based incentives or bundles are effective
- 📌 **Delivery timing** reveals potential misalignment between real vs. promised shipping — investigate scheduling logic

## 📄 **Page 2: Time Trends & Performance Over Time**

### 🎯 Goal

Visualize how order volume, sales, profit and delivery behavior evolve over time — useful for identifying seasonality, growth patterns or operational issues.

### 🎛 Filters

Includes same filters from Page 1, for consistency:
- 📅 `Date` (from `dim_date`)
- 📦 `Category` (from `dim_product`)
- 🌎 `Country` (from `dim_order_location`)
- 👥 `Segment` (from `dim_customer`)

### 📊 KPIs on the Left

- **KPIs that support time-series focus, like:**
  - 💰 Total Revenue
  - 🧾 Average Orders

These KPIs are tied to key insights from the earlier **SQL & DAX analysis**. They are filter-responsive for slicing by time, geography or category.

### 📈 Visual Elements

#### _1. Orders Trend_

- **X-axis:** Month/Year (`dim_date[year_month]`)
- **Y-axis:** Total Orders

🔍 Shows growth patterns, seasonal peaks (holidays, Q4 spikes).

#### _2. Profit Trend_

- **X-axis:** Month/Year (`dim_date[year_month]`)
- **Y-axis:** Total Profit

🔍 See if profit follows orders or diverges — indicates rising costs.

#### _3. Avg. Shipping Delay Over Time_

- **X-axis:** Month/Year (`dim_date[year_month]`)
- **Y-axis:** Average Shipping Delay
- **Secondary Y-axis:** Average Shipping Delay

🔍 Is delivery improving over time? Are the spikes during busy months?

#### _4. YoY Comparison_

- **X-avis:** Month (from `dim_date[year_month]` with **Date Hierarchy** in **Month**)
- **Y-axis:** Total Revenue
- **Legend:** Year (from `dim_date[year]`)

🔍 Helps benchmarking monthly performance YoY.

### 🧠 Insights to Highlight

- Do orders follow a consistent **monthly volume**? Are there peak/dip months?
- Is **profit growing** at the same pace as **revenue** or **order volume**?
- How does **average shipping delay** fluctuate across time?
- Any **anomalies** in Year-over-Year patterns (e.g. sudden drop in Q4 2017 and 2018)?

### 📌 Insights Taken

- **Shipping delays** show a **slight upward trend** in early 2015, early 2016, mid 2016 and late 2017. Possibly due to operational constraints or rising demand on Q4 2017 — should be further investigated.
- **YoY performance** is consistent until late 2017 when a **noticeable revenue drop** occurs — needs to be investigated.
- **Order volume** remained stable for most of the period, with a **sharp rise in late 2017**, suggesting a surge of campaign.
- Despite higher orders, **profit dropped exponentially in late 2017**, hinting at either **costlier fulfillment** or **heavier discounts**.

## 📄 Page 3: Customer & Product Intelligence

### 🎯 Goal

Used to uncover who drives the business — segmenting **customer behaviors** and identifying **top-performing products/categories**.

### 🎛 Filters

Includes same filters as Page 1, giving further consistency:  
- 📅 `Date` (from `dim_date`)
- 📦 `Category` (from `dim_product`)
- 🌎 `Country` (from `dim_order_location`)
- 👥 `Segment` (from `dim_customer`)

### 📊 KPIs on the Left

- **Financial KPIs like:**
    - 🧾 Total Customers
    - 💹 Top Customers Revenue %

- **Customer & Order KPIs like:**
    - 👤 New vs Returning Customers
    - 📊 Top Categories by Revenue

As with previous pages, these KPIs are also tied to key insights from previously **SQL & DAX analysis** and are filter-reponsive. Can be filtered either by time, geography or category.

### 📈 Visual Elements

#### _1. Top Customers_

- **X-axis:** Total Revenue
- **Y-axis:** Customer ID (`dim_customer[customer_id]`)
- **Tooltip:** Customer Name (`dim_customer[customer_name]`)
- **Top N:** Uses Top N filtering to highlight Top 10 customers by revenue

🔍 Highlights top 10 customers that drive the most revenue and sorts them.

#### _2. Top Products_

- **X-axis:** Total Revenue
- **Y-axis:** Product Name (`dim_product[product_name]`)
- **Top N:** Uses Top N filtering to highlight Top 10 products by revenue

🔍 Highlights the top 10 products sold sorted by revenue.

#### _3. Average Cart Size by Segment_

- **X-axis:** Customer Segment (`dim_customer[segment]`)
- **Y-axis:** Average Cart Size

🔍 Offers segmentation of average cart size by customer segment sorted.

#### _4. Category vs Profit Margin_

- **X-axis:** Category ID (`dim_category[category_id]`)
- **Y-axis:** Profit Margin (%)
- **Legend:** Category Name (`dim_category[category_name]`)

🔍 Presents the relationship between category and profit margin.

#### _5. New vs Returning Customers_

- **X-axis:** Month/Year (`dim_date[year_month]`)
- **Y-axis:** Total Customers
- **Legend:** Customer Type (`customer_first_orders[customer_type]`)

🔍 Displays count of new customers vs returning customers over time.

### 🧠 Insights to Highlight

- Do a **few customers** contribute a **disproportional amount** of revenue?
- Do **certain products** drive a **disproportional amount** of revenue? Does **more revenue** equals **more orders**?
- How do **customer segment** differ in behavior?
- Are **certain categories profitable** but undersold?
- Business is driven by **loyal customers** or do **new customers** offer the business **more orders & revenue**?

### 📌 Insights Taken

- **Top 10 customers** aren't the main source of revenue, contributing only for 0,25% of revenue. There is no need for further **loyalty incentives** and **retention focus**, however, they can be used for means of **improving customer satisfaction**.
- Certain **products anmd categories** drive significantly higher revenue — showing potential for **cross-sell**, **bundling** or **premium positoning**.
- **Average car size** are roughly the same throughout all segments. **Consumer** segment is keeping up with both **Corporate** and **Home Office** segments, indicating successful **B2C bundling strategies**.
- **Profit margins vary across categories**, highlighting areas where **sales volume don't translate into profitability** — it calls for pricing or operational review in some categories (e.g. `Strength Training`, `As Seen on TV!`, `Men's Clothing`).
- **New customer count increases** in later months, while returnin customers tend to remain stable throughout the whole year, slightly declining in the last quarter — this suggests **market saturation** or need for **acquisition strategies**.

## 📄 Page 4: Late Delivery Trends

### 🎯 Goal

This page is used for insights into **late deliveries** patterns over time. Gives visibility on **logistics performance** and how they can be affected by **country and market**, **shipping mode** or even **categories**.

### 🎛 Filters

Once again the filters used are the same as previous pages, keeping consistency and tying pages together:  
- 📅 `Date` (from `dim_date`)
- 📦 `Category` (from `dim_product`)
- 🌎 `Country` (from `dim_order_location`)
- 👥 `Segment` (from `dim_customer`)

### 📊 KPIs on the Left

- **Financial KPIs like:**
    - 🧾 Total Orders

- **Order KPIs like:**
    - ⏱ Late Delivery Rate
    - 🚚 Late Delivery Orders
    - 📦 Average Shipping Delay

These KPIs are directly tied to **SQL & DAX analysis** that was made previously and give insights on late deliveries. They are filter-ready by time, geography, category or customer segment.

### 📈 Visual Elements

#### _1. Late Delivery Rate Over Time_

- **X-axis:** Month/Year (`dim_date[year_month]`)
- **Y-axis:** Late Delivery Rate (%)

🔍 Used to observe seasonality or operational slowdowns (e.g. holidays, supply chain bottlenecks).

#### _2. Late Deliveries by Country_

- **Location:** Country (`dim_order_location[order_country]`)
- **Gradient:** Uses a conditional color scale to highlight regional late deliveries rate

🔍 This maps spots worldwide region bottlenecks or distribution issues.

#### _3. Late Delivery Rate by Shipping Mode_

- **X-axis:** Shipping Mode (`dim_shipping[shipping_mode]`)
- **Y-axis:** Late Delivery Rate (%)

🔍 The purpose of this chart is to identify the least reliable shipping methods and which ones need more logistics attention.

#### _4. Late Delivery Rate by Market_

- **X-axis:** Market (`dim_order_location[market]`)
- **Y-axis:** Late Delivery Rate (%)

🔍 The chart is used to highlight logistics reliability by market.

#### _5. Worst Delivery Rate Categories_

- **Category:** Category Name (`dim_category[category]`)
- **Values:** Late Delivery Rate (%)
- **Top N:** Uses Top N filtering to highlight the 10 worst categories by late delivery rate

🔍 This treemap highlights the worst performing categories by late delivery rate. Informs of what category might need best logistics strategies to lower late delivery rates.

### 🧠 Insights to Highlight

- Are delays **increasing over time**? Or do they show **seasonal patterns**?
- Do **some shipping modes underperform consistently**? Which ones are **most reliable**?
- Are late deliveries concentrated in **specifict countries or markets**?
- Do **certain product categories** tend to **experience more delivery issues**?
- Can we improve **customer satisfaction** by **revisiting fulfillment schedules** or **shipping promises**?

### 📌 Insights Taken

- **Over 54% of orders** are flagged as late — indicating either **logistical inefficiency** or **inaccurate promised delivery dates**.
- **First Class shipping** shows the **worst performance**, whereas **Same Day** and **Standard Class** are comparatively more reliable.
- **Markets like Pacific Asia and Europe** have slightly higher delivery rates — however, the increased values are redundant, hinting for **supply chain issues** and **operational issues worldwide**.
- Certain **product categories** (e.g. Golf Bags, Lacrosse) have high late delivery rates — which may relate to **supply chain complexity** or **supplier locations**.
- Despite high late delivery rate, the **average shipping delay (in days)** is relatively low (~0.5 days), implying the issue might lie in **overly strict delivery promises**.

## 📄 Page 5: Profitability & Efficiency Analysis

### 🎯 Goal

Uncover what drives profitability and assess how discounts, shipping delays and customer behavior affect overall efficiency. Helps surface high-revenue areas that may not be profitable and highlight operational inefficiencies.

### 🎛 Filters

Same filters for consistency:
- 📅 `Date` (from `dim_date`)
- 📦 `Category` (from `dim_product`)
- 🌎 `Country` (from `dim_order_location`)
- 👥 `Segment` (from `dim_customer`)

### 📊 KPIs on the Left

- **Financial KPIs like:**
    - 📊 Profit Margin %
    - 💹 Top Category by Profit
    - 🏷️ Total Discounts
    - 📉 Negative Profit Orders

- **Operational KPIs like:**
    - 💸 Average Discount
    - 💰 Average Profit

- **Conditional KPIs like:**
    - ⬆️ Profit Gain
      - `Total Profit` filtered on `fact_orders[revenue_per_order] > 0` 
    - ⬇️ Profit Loss
      - `Total Profit` filtered on `fact_orders[revenue_per_order] < 0`

KPIs are taken from the usage of **SQL & DAX analysis** like before and can be filtered either by time, geography, category or segment.

### 📈 Visual Elements

#### _1. Average Discount by Product_

- **X-axis:** Product Name (`dim_product[product_name]`)
- **Y-axis:** Average Discount
- **Top N:** Uses Top N filtering to highlight Top 10 products with highest average discount

🔍 Flags top products being heavily discounted — could be impacting profitability.

#### _2. Revenue vs Profit Margin_

- **X-axis:** Total Revenue
- **Y-axis:** Profit Margin (%)
- **Legend:** Category Name (`dim_category[category]`)

🔍 High-revenue and low-margin categories stand out — makes it easier to spot top performing or underperforming categories, either by revenue or by profit margin.

#### _3. Profit by Country_

- **Location:** Country (`dim_order_location[order_country]`)
- **Gradient:** Uses a conditional color scale to highlight regional profit

🔍 Used to spot regional efficiency and profitability evaluation.

#### _4. Negative Profit Orders_

- **Columns:**
    - Order ID (`fact_orders[order_id]`)
    - Quantity (`fact_orders[item_quantity]`)
    - Profit (`Total Profit`)
    - Discount (`Total Discount`)
    - Revenue (`Total Revenue`)
    - Category (`dim_category[category]`)
- **Filter:** Uses filtering to highlight orders with negative profit
- **Sort:** Sorts orders from lowest to highest profit

🔍 Shows values of loss-making orders

### 🧠 Insights to Highlight

-  Are we **selling high-volume** and **low-margin products**?
-  Are **discounts driving volume** or just **killing margins**?
-  Do certain regions or categories **have lower efficiency**?
-  How **many orders** are we **losing money on**?
-  Can we better **balance volume**, **price** and **profit**?

### 📌 Insights Taken

- **Loss-generating orders aresubstantial**, with over **14000 orders having negative profit** — a red flag that should be addressed through pricing or discount policies.
- The **Fishing category** ranks highest in profit contribution, this suggests strong margin and sales synergy in the category. It can be used to find a better model for the other categories.
- **Computers** dominate the loss-making order list, hinting at potential **over-discounting** or even **low markup** in this category.
- Certain countries show **high revenue but poor profit**, this indicates that **operational costs**, **fulfillment logistics** or **regional discounting** may be eating at margins.
- **Average product discount is above 20**, with some SKUs **reaching 200-250** — these seem healthy at first, but need more investigation as average profit seems too low, being just slightly either than the average discount.
- Overall **profit margin remains healthy (~12%)**, but deeper segmentation reveals there are **significant inefficiencies hidden behind aggregate numbers**.

## 📄 Page 6: Executive Summary & Strategic Insights

### 🎯 Goal

Provides a strategic overview of the most impactful insights uncovered across the analysis — identifying areas of opportunity, concern and actions to take.  
Rather than introducing more granular analysis, this page **summarizes**, **prioritizes** and **visualizes impact areas** for decision-makers.

### 🎛 Filters

Includes basic filtering like previous pages for consistency:
- 📅 `Date` (from `dim_date`)
- 📦 `Category` (from `dim_product`)
- 🌎 `Country` (from `dim_order_location`)
- 👥 `Segment` (from `dim_customer`)

### 📊 KPIs on the Left

- **Strategic KPIs like:**
    - 📊 Profit Margin %
    - ⏱ Late Delivery %
    - 🏷️ Discount Orders %
    - 👤 Top Customer Revenue %

### 📈 Visual Elements

#### _1. Action Radar_

- **Three-column layout using text boxes:**
    - ⚙️ What Needs to Improve
    - ✅ What Needs to Grow
    - 🚨 What Needs to Be Fixed

🔍 Helps summarize business priorities and guide stakeholder focus

#### _2. Market Performance_

- **Rows:**
    - Market (`dim_order_location[market]`)
- **Values:**
    - Profit Margin (%)
    - Late Delivery Rate (%)

🔍 Heatmap-style table comparing performance across markets to identify regional outliers.

#### _3. Categories vs Metrics_

- **Rows:**
    - Category (`dim_category[category]`)
    - Product Name (`dim_product[product_name]`)
- **Values:**
    - Profit Margin (%)
    - Discount Orders Rate (%)
    - Late Delivery Rate (%)

🔍 Identifies tradeoffs of Profit, Discount and Late Deliveries throughout all categories, enables root-cause targeting at category level.

#### _4. Insights Cards_

- **Five text boxes including summary of:**
    - 💰 Revenue & Profit
    - 🚚 Shipping & Logistics
    - 📦 Product Performance
    - 👥 Customer Behavior
    - 🌍 Geography

🔍 Five high-level business summaries using cards and text visuals.

### 🧠 Insights to Highlight

- Are we maintaining a healthy balance between **profit and volume**?
- Are **logistics bottlenecks** (e.g. high delays) affecting customer experience?
- Are **loss leaders** hurting profitability?
- Which **categories or regions** need pricing/logistics attention?
- What are the **biggest levers** to improve bottom line?

### 📌 Insights Taken

- 💰 **Revenue & Profit Performance**  
    >- Despite generating **$33M in revenue** and maintaining a **12% margin**, profit leakage is evident in **~14K unprofitable orders**, concentrated heavily in the **Computer category**. This suggests an urgent need to **review pricing strategies** or **discount rules** in high-revenue, low-margin categories.
    >
    >- Additionally, the **top 10 customers contribute ~0.25% of total revenue** highlighting that while the very top customers contribute a small fraction of overall revenue, there isn't a significant reliance on a small subset of accounts at the very top. Even with this small contribution, it's still crucial to prioritize their **retention**. While their individual revenue impact might be low, losing several of them could still represent a noticeable decline in overall revenue and potentially signal underlying issues.

- 🚚 **Shipping & Logistics Risk**
    >- The **late delivery rate exceeds 54%**, well above standard benchmarks. While the **average delay (0.57 days)** might seem minimal, this aggregate hides the fact that **"First Class" shipping shows ~95% delays**, indicating a severe mismatch between **promised and actual delivery performance**. This calls for a **reevaluation of service level agreements (SLAs)** and possibly **realignment of delivery modes**.

- 📦 **Product Profitability Concerns**
    >- **Computers**, while popular, are acting as **loss leaders**, appearing frequently in **top profit loss orders**. These may be **intentional promotions** or **pricing rule flaws**. In contrast, **Fishing products** consistently yield **high margins** as well as **high revenue and profit**, making them ideal for **growth campaigns and upselling** strategies.

- 👤 **Customer Behavior & Segmentation**
    >- Returning customers dominate the customer base with a **1.4 : 1 ration over new customers**, suggesting good retention. However, **average order size remains high (5.84 items per order)**, which may be driven by **bundled promotions or bulk buying behavior** — both of which can be optimized further.
    >- This also provides a foundation for a more advanced **RFM segmentation strategy** or **lifetime value modeling**, particularly to convert high cart-size behavior into a long-term retention.

- 🌍 **Geographic Efficiency**
    >- Markets like **Pacific Asia and Europe** show **highest late delivery rates** while having the **worst margins**, signaling a potential combination of **logistics bottlenecks** and **underperforming product mixes**. These regions should be prioritized for **process improvements** or **pricing adjustments**.
    >- Conversely, **North America (US/Canada)** remains the most **revenue-dense** and **operationally stable**, suggesting that proven logistics and category strategies here could be **replicated or adapted elsewhere**.

## 📊 Final Highlights & Strategic Takeaways

Across the full supply chain and customers analytics journey, several **key themes** have emerged:

🔍 **1. Profitability Isn't Tied to Volume**  
- High revenue doesn't always translate to strong margins — categories like **Computers** and regions like **Europe** show significant revenue but weak profit performance. Discount strategies should be re-evaluated.

📦 **2. Shipping & Fulfillment Is a Bottleneck**
- With a **~54% late delivery rate**, especially on **First Class shipping**, logistics is a major operational concern. Despite low average delays, reliability and expectation mismatches impact customer trust.

🛍️ **3. Strong Customer Behavior Foundations**
- The business has solid **retention rates**, a healthy **average cart size** and **loyal top-spending customers**. These are strong foundations for **personalization**, **loyalty programs** or **lifetime value growth strategies**.

🌍 **4. Market Expansion Opportunities**
- North America dominates in performance, but **regions like Asia and Europe** show both room for growth and operational inefficiencies — warranting further market-specific review.

💡 **5. Action Prioritization is Clear**
- From the Action Radar, we know what needs **urgent fixing** (e.g. shipping delays, loss-making products), what should be **monitored or improved** and what should be **expanded strategically**.

> In the end, the dataset reflects a business that's mature in **volume** but still has **high-impact optimization opportunities** in **logistics**, **pricing** and **customer value extraction**.